In [185]:
import Quandl
import pandas as pd
import numpy as np
import datetime

In [186]:
start_date = datetime.datetime(2010,1,1)
end_date = datetime.datetime(2016,1,1)
market = 'SPY'
test_ticker = 'AAPL'
window = 100

In [187]:
nasdaq_url = 'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download'
df_tickers = pd.read_csv(nasdaq_url)

In [188]:
# process tickers
df_tickers.rename(columns={ 'Symbol': 'ticker', 'Name': 'name', 'LastSale': 'last_price', 'MarketCap': 'market_cap',
                           'IPOyear': 'ipo_year', 'Sector': 'sector', 'Industry': 'industry'}, inplace=True)

In [189]:
df_tickers['ipo_year'] = df_tickers['ipo_year'].convert_objects(convert_numeric=True)
df_tickers.dropna(subset=['ipo_year'], inplace=True)
df_tickers = df_tickers[(df_tickers['market_cap'] > 1e9) & (df_tickers['ipo_year'] < 2010)]

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if __name__ == '__main__':


In [190]:
df_market = Quandl.get('GOOG/NYSE_'+market)
df_market.rename(columns={'Close': market}, inplace=True)
df_market[market] = df_market[market].pct_change()
df_master = df_market[(df_market.index >= start_date) & (df_market.index < end_date)][[market]]

In [191]:
tickers = list(df_tickers['ticker'][:10])
# tickers = ['AAPL', 'MSFT']


In [194]:
for t in tickers:
    try: 
        df_stock = Quandl.get('WIKI/'+t, start_date='2010-01-01', end_date="2005-12-31")
    except:
        print ('{} not found in Quandl '.format(t))
        continue
    df_master[t] = df_stock[(df_stock.index >= start_date) & (df_stock.index < end_date)]['Adj. Close']
    df_master[t] = df_master[t].pct_change()
    print('{} successfully pulled'.format(t))

ABAX successfully pulled
ACAD successfully pulled
ADBE successfully pulled
ADTN successfully pulled
AEIS successfully pulled
AGNC successfully pulled
AKAM successfully pulled
ALXN successfully pulled
ALGN successfully pulled
ALKS successfully pulled


In [195]:
df_master.dropna(subset=['ABAX'], inplace=True)

In [196]:
tickers = list(set.intersection(set(df_master.columns), tickers)) # update tickers list
# tickers_ch = [t+'_ch' for t in tickers]

In [197]:
df_master.to_pickle('df_master.pkl')

## Calcs

In [198]:
covs = df_master[tickers].rolling(window=window).cov(df_master[market], pairwise=True)
var = df_master[market].rolling(window=window).var()
df_beta = covs.div(var,axis=0)

In [199]:
df_beta.to_pickle('df_beta.pkl')

In [200]:
today = df_beta.index.max()
sr_beta_today = df_beta.loc[df_beta.index.max()]
df_tickers = df_tickers[df_tickers['ticker'].isin(tickers)]
df_tickers = df_tickers.set_index('ticker')
df_tickers['ticker'] = df_tickers.index
df_tickers['beta'] = sr_beta_today

In [201]:
ticker_fields = ['ticker', 'name', 'beta', 'ipo_year', 'market_cap', 'sector', 'industry', 'last_price']
df_tickers = df_tickers[ticker_fields]
df_tickers[ticker_fields].to_pickle('df_tickers.pkl')